In [7]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree



# Para realizar la regresión lineal y la evaluación del modelo
# -----------------------------------------------------------------------
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from category_encoders import TargetEncoder
from sklearn.ensemble import GradientBoostingRegressor



from sklearn.model_selection import KFold,LeaveOneOut, cross_val_score


from sklearn.preprocessing import StandardScaler

from tqdm import tqdm

import joblib  # Importar la biblioteca para guardar modelos



# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')
import os 
import sys
sys.path.append("../../")
from src import soporte_ajuste as sa

pd.options.display.float_format = "{:,.2f}".format 
pd.set_option("display.max_columns", None)

In [8]:
df=pd.read_csv("../../datos/madrid_preprocesado.csv", index_col=0)
df

,price,propertyType,operation,size,exterior,rooms,bathrooms,address,province,municipality,country,showAddress,distance,status,newDevelopment,hasStaging,floor,district,hasLift,priceByArea_knn,price_robust,rooms_robust,bathrooms_robust,priceByArea_knn_robust
0,710.00,697.55,692.91,55.00,True,2,1,695.14,692.91,705.63,692.91,True,"11,596.50",693.87,694.40,693.72,696.09,693.92,701.99,12.00,0.00,0.00,0.00,0.00
1,710.00,697.55,692.91,55.00,True,2,1,695.14,692.91,705.63,692.91,False,"11,596.50",693.87,694.40,693.72,696.09,693.92,701.99,12.00,0.00,0.00,0.00,0.00
2,710.00,697.55,692.91,55.00,True,2,1,695.14,692.91,705.63,692.91,False,"11,596.50",693.87,694.40,693.72,696.09,693.92,701.99,12.00,0.00,0.00,0.00,0.00
3,710.00,697.55,692.91,55.00,True,2,1,695.14,692.91,705.63,692.91,False,"11,596.50",693.87,694.40,693.72,696.09,693.92,701.99,12.00,0.00,0.00,0.00,0.00
4,684.00,689.81,692.91,45.00,True,0,1,691.75,692.91,705.63,692.91,True,"10,656.00",693.87,694.40,693.72,671.74,645.66,673.70,13.30,-0.40,-2.00,0.00,0.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,640.00,697.55,692.91,69.00,True,2,1,686.03,692.91,686.03,692.91,True,"29,311.00",693.87,694.40,693.72,696.09,645.66,701.99,9.00,-1.08,0.00,0.00,-0.50
354,700.00,697.55,692.91,54.00,True,1,1,693.84,692.91,696.16,692.91,False,"11,656.00",693.87,694.40,693.72,701.70,688.60,701.99,13.00,-0.15,-1.00,0.00,0.17
355,700.00,697.55,692.91,59.00,True,1,1,693.84,692.91,705.63,692.91,True,"7,529.00",693.87,694.40,693.72,697.47,701.64,701.99,12.00,-0.15,-1.00,0.00,0.00
356,700.00,697.55,692.91,75.00,True,2,1,693.84,692.91,699.61,692.91,True,"30,359.00",693.87,694.40,693.72,701.70,693.84,689.68,9.00,-0.15,0.00,0.00,-0.50


In [9]:
x= df.drop(columns="price")
y= df[["price"]]

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size= 0.7, random_state=42)

In [10]:
if isinstance(y, pd.DataFrame):
    y = y.iloc[:, 0]  # Convertir a Serie usando la primera columna

# Dividimos los datos en conjuntos de entrenamiento (train) y prueba (test)
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.8, random_state=42
)

# Verificar las formas de los conjuntos de datos resultantes
print(f"Forma de X_train: {X_train.shape}")
print(f"Forma de X_test: {X_test.shape}")
print(f"Forma de y_train: {y_train.shape}")
print(f"Forma de y_test: {y_test.shape}")

# Crear y entrenar el modelo
gbr_model = GradientBoostingRegressor(random_state=42, n_estimators=100, learning_rate=0.1)
gbr_model.fit(X_train, y_train)

# Predicción y evaluación
y_pred_gbr = gbr_model.predict(X_test)

# Calcular el RMSE
rmse_gbr = np.sqrt(mean_squared_error(y_test, y_pred_gbr))

# Calcular el coeficiente R^2
r2_gbr = r2_score(y_test, y_pred_gbr)

# Mostrar resultados
print("Precio real vs Predicción:")
display(pd.DataFrame({'Real': y_test.values, 'Predicción': y_pred_gbr}).head(2))  # Mostrar las primeras 2 filas
print(f"RMSE de Gradient Boosting: {rmse_gbr:.2f}")
print(f"R^2 de Gradient Boosting: {r2_gbr:.2f}")

joblib.dump(gbr_model, "modelo_madrid.pkl")
print("Modelo guardado exitosamente como gbr_model.pkl")

Forma de X_train: (70, 23)
Forma de X_test: (282, 23)
Forma de y_train: (70,)
Forma de y_test: (282,)
Precio real vs Predicción:


,Real,Predicción
0,700.00,704.79
1,680.00,641.26


RMSE de Gradient Boosting: 5.24
R^2 de Gradient Boosting: 0.99
Modelo guardado exitosamente como gbr_model.pkl


<div style="background-color: #FFBB33; padding: 10px; border-left: 6px solid #000080; color: black; border-radius: 10px;">
RMSE : 5.24  

R^2: 0.99
</div>

<div style="background-color: #FBFB; padding: 10px; border-left: 6px solid #000080; color: black; border-radius: 10px;">
Conclusión:  

La tercera métrica es la mejor, ya que tiene el menor error (RMSE más bajo) y mantiene un excelente (R²=0.99).
</div>